## 01 libary & paramiter

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pathlib
import db_connect
from datetime import datetime, timedelta
import os
from openpyxl import load_workbook
import shutil

# Set parameters
bu = 'ssp'
date = '20250101'

table_stk = 'stk'
table_var = 'var'

rpname_stk = 'STK2'
rpname_var = 'VAR2'

sheet_name_stk = 'detailsku'
sheet_name_var = 'variancelocation'

column_stk = 'A:AC'
column_var = 'A:V'

#connect to database
connect_db3 = create_engine(db_connect.db_url_pstdb3)
connect_db = create_engine(db_connect.db_url_pstdb)


# Set file path
user_path = pathlib.Path.home()
f_path = user_path / 'Documents' / 'soh' / 'report3' / str.upper(bu) / 'Store' 

excel_files = [
    f for f in os.listdir(f_path)
    if f.lower().endswith(('xls','.xlsx', '.xlsm')) and os.path.getsize(f_path / f) > 0
]

print(f"{len(excel_files)} Files")

# get data from stk db3
stk2_db3 = f"""
SELECT distinct
    store, cntdate,skutype, rpname,'recheck' as recheck
FROM {bu}_{table_stk}_this_year
where rpname = '{rpname_stk}'
"""
stk2_db3 = pd.read_sql(stk2_db3, connect_db3)

# get data from plan db
plan_db = f"""
SELECT 
    stcode as store, cntdate,branch
FROM planall2
where atype = '3F'
    and cntdate >= '{date}'
    and bu = '{bu.upper()}'
"""
plan_db = pd.read_sql(plan_db, connect_db)

# get data from var db3
var2_db3 = f"""
SELECT distinct
    store, cntdate,skutype, rpname,'recheck' as recheck
FROM {bu}_{table_var}_this_year
where rpname = '{rpname_var}'"""
var2_db3 = pd.read_sql(var2_db3, connect_db3)

14 Files


## 02 rename sheet

In [ ]:
for fname in excel_files:
    file_path = f_path / fname
    try:
        wb = load_workbook(file_path)
        changed = False
        for sheet in wb.sheetnames:
            new_name = sheet.lower().replace(" ", "")
            if new_name != sheet:
                wb[sheet].title = new_name
                changed = True
                print(f"✅{fname}: '{sheet}' ➝ '{new_name}'")
        if changed:
            wb.save(file_path)  # บันทึกทับไฟล์เดิม
    except Exception as e:
        print(f"❌ Error {fname}: {e}")

## 03 upload STK2 to db

In [ ]:
for file in excel_files:
    file_path = os.path.join(f_path, file)
    try:
        xls = pd.ExcelFile(file_path)
        if sheet_name_stk in xls.sheet_names:
            df = pd.read_excel(file_path, sheet_name=sheet_name_stk, usecols=column_stk, dtype=str)

            df.columns = df.columns.str.lower()

            file_parts = os.path.splitext(file)[0].split('_')
            if len(file_parts) == 5:
                cols1, cols2, cols3, cols4, cols5 = file_parts
            else:
                cols1, cols2, cols3, cols4, cols5 = None, None, None, None, None

            # เพิ่มคอลัมน์ใหม่
            df['cntdate'] = cols4
            df['rpname'] = rpname_stk

            # ถ้า COUNTNAME ไม่มีค่า (NaN) ให้ fill ด้วย '' เพื่อป้องกัน error
            df['sku'] = df['sku'].fillna('')

            # สร้างคอลัมน์ skutype
            df['skutype'] = df['countname'].apply(lambda x: 'Credit' if x[1:2] == 'B' else 'Consign')

            # กรองตาม STORE
            df = df[df['store'] == cols3]

            # join plan db
            df = df.merge(plan_db[['store', 'cntdate','branch']],
                          on=['store', 'cntdate'],
                          how='left')
            # keep only rows with branch **not null**
            df = df[df['branch'].notna()]

            # join stk db3
            df = df.merge(stk2_db3[['store', 'cntdate', 'skutype', 'rpname','recheck']],
                          on=['store', 'cntdate', 'skutype', 'rpname'],
                          how='left')
            # keep only rows with recheck **is null**
            df = df[df['recheck'].isna()]

            df = df.drop(columns=['branch', 'recheck'])

            df.to_sql(f'{bu}_{table_stk}_this_year', connect_db3, if_exists='append', index=False)

            print(f"✅Processed & inserted {file} with {len(df)} rows to {bu}_{table_stk}_this_year ({excel_files.index(file)+1}/{len(excel_files)})")
        else:
            print(f"❌sheet not found in {file}")
    except Exception as e:
        print(f"❌Error processing {file}: {e}")

## 04 upload VAR2 to db

In [2]:
for file in excel_files:
    file_path = os.path.join(f_path, file)
    try:
        xls = pd.ExcelFile(file_path)
        if sheet_name_var in xls.sheet_names:
            df = pd.read_excel(file_path, sheet_name=sheet_name_var, usecols=column_var, dtype=str)

            df.columns = df.columns.str.lower()

            file_parts = os.path.splitext(file)[0].split('_')
            if len(file_parts) == 5:
                cols1, cols2, cols3, cols4, cols5 = file_parts
            else:
                cols1, cols2, cols3, cols4, cols5 = None, None, None, None, None

            # เพิ่มคอลัมน์ใหม่
            df['cntdate'] = cols4
            df['rpname'] = rpname_var

            # ถ้า COUNTNAME ไม่มีค่า (NaN) ให้ fill ด้วย '' เพื่อป้องกัน error
            df['sku'] = df['sku'].fillna('')

            # สร้างคอลัมน์ skutype
            df['skutype'] = df['countname'].apply(lambda x: 'Credit' if x[1:2] == 'B' else 'Consign')

            # กรองตาม STORE
            df = df[df['store'] == cols3]

            # join plan db
            df = df.merge(plan_db[['store', 'cntdate','branch']],
                          on=['store', 'cntdate'],
                          how='left')
            # keep only rows with branch **not null**
            df = df[df['branch'].notna()]

            # join stk db3
            df = df.merge(var2_db3[['store', 'cntdate', 'skutype', 'rpname','recheck']],
                          on=['store', 'cntdate', 'skutype', 'rpname'],
                          how='left')
            # keep only rows with recheck **is null**
            df = df[df['recheck'].isna()]

            df = df.drop(columns=['branch', 'recheck'])

            df.to_sql(f'{bu}_{table_var}_this_year', connect_db3, if_exists='append', index=False)

            xls.close()

            shutil.move(file_path, user_path / 'Documents' / 'soh' / 'report3' / 'Processed' / file)    

            print(f"✅Processed & inserted {file} with {len(df)} rows to {bu}_{table_var}_this_year ({excel_files.index(file)+1}/{len(excel_files)})")
        else:
            print(f"❌sheet not found in {file} ({excel_files.index(file)+1}/{len(excel_files)})")
    except Exception as e:
        print(f"❌Error processing {file}: {e}")

✅Processed & inserted Report3_SSP_80034_20250701_0.xlsx with 0 rows to ssp_var_this_year (1/14)
✅Processed & inserted Report3_SSP_80053_20250624_0.xlsx with 0 rows to ssp_var_this_year (2/14)
✅Processed & inserted Report3_SSP_80093_20250624_0.xlsx with 0 rows to ssp_var_this_year (3/14)
✅Processed & inserted Report3_SSP_80101_20250805_0.xlsx with 0 rows to ssp_var_this_year (4/14)
✅Processed & inserted Report3_SSP_80136_20250519_0.xlsx with 0 rows to ssp_var_this_year (5/14)
✅Processed & inserted Report3_SSP_80137_20250625_0.xlsx with 0 rows to ssp_var_this_year (6/14)
✅Processed & inserted Report3_SSP_80204_20250617_0.xlsx with 0 rows to ssp_var_this_year (7/14)
✅Processed & inserted Report3_SSP_80226_20250819_0.xlsx with 0 rows to ssp_var_this_year (8/14)
✅Processed & inserted Report3_SSP_80249_20250624_0.xlsx with 0 rows to ssp_var_this_year (9/14)
✅Processed & inserted Report3_SSP_80303_20250626_0.xlsx with 0 rows to ssp_var_this_year (10/14)
✅Processed & inserted Report3_SSP_80312